In [ ]:
# Just the normal prequisites for using matplotlib and numpy in a Jupyter notebook
%matplotlib inline
# Use the svg backend, in my opinion it just makes better looking plots
%config InlineBackend.figure_format = 'svg'

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging( logging_level='DEBUG')

import os
import sys
import IPython
from scipy import signal
import scipy.fftpack
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Plot.BodeDiagram import bode_diagram
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import unittest

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging()

directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)

directory_path = Path(os.path.abspath('')).resolve()


![module](https://img.shields.io/badge/module-shin%20ey-yellow) ![status](https://img.shields.io/badge/status-work%20in%20progress-orange)

## *shin ey*

<a href="https://photos.app.goo.gl/pg6iZUC32rTKX5LC7"><img src="https://spielhuus.github.io/elektrophon/images/kontrast-logo-tmb.jpg" height="300px" align="right"></a>

***table of contents***

* [*about*](#about)
* [*construction*](#construction)
* [*calibration*](#calibration)
* [*usage*](#usage)
* [*credits*](#credits)
* [*links*](#links)
* [*changelog*](#changelog)
  
<br/><br/><br/><br/>

## *about*

pegel is a voltage controlled amplifier (vca). there are various designs to implement a vca. rod elliott (ESP) has an article on vca techniques [[1][1]]. diy synthesizer modules are usualy designed with an integrated vca chip [[2][2]]  or with a differential amplifier [[3][3]]. the integrated vca chips are either obsolete or rather expensive. the differential amplifier can be built with transistors only. the design has some downsides. even in the simulation the result is not symetryc and has a dc offset from the control voltage. this needs a lot of trimming to get an accurate result. but the biggest downside is, that this design can not do proper amplitude modulation (am). when the carrier signal goes below zero the base signal is completely muted. 

### input mixer

this  is  a pasive input mixer.


In [ ]:
from sympy import symbols, sqrt, exp, diff, integrate, pprint, simplify, Eq, preview, pi, solve

from sympy import init_printing
#init_printing(use_latex='mathml')

R, C, fc = symbols('R,C,fc', real=True)
lp_f = 1 / ( 2 * pi * R * C)
Eq( lp_f, fc)


In [ ]:
R1, R2, C1, C2, fc = symbols('R1,R2,C1,C2,fc')
lp_f = 1 / ( 2 * pi * sqrt(R1 * R2 * C1 * C2))

Eq(fc, lp_f)


In [ ]:
r1 = 10@u_kΩ
r2 = 10@u_kΩ
c1 = 0.082@u_uF
c2 = 0.082@u_uF

Eq(fc, lp_f.subs(R1, r1).subs(C1, c1).subs(R2, r2).subs(C2, c2).subs(pi,math.pi))


In [ ]:
r1 = 10@u_kΩ
r2 = 10@u_kΩ
c1 = 4.7@u_nF
c2 = 4.7@u_nF

Eq(fc, lp_f.subs(R1, r1).subs(C1, c1).subs(R2, r2).subs(C2, c2).subs(pi,math.pi))

## low pass filter


In [ ]:
#load the input mixer schema
kicad_netlist_path = directory_path.joinpath('main', 'LowPass200.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

#simulate input
schema = parser.build_circuit(ground=5)
schema.include(spice_library['4069UB'])
schema.V('1', '+15V', schema.gnd, 'DC 15')
schema.V('2', '-15V', schema.gnd, 'DC -15')
#schema.V('3', 'NC_01', schema.gnd, 'DC 0V AC 5V SINE(0 5V 1k)')
schema.V('3', 'NC_01', schema.gnd, 'DC 0V AC 5V PULSE(-5V 5V 1u 1u 1u 2m 4m)')

simulator = schema.simulator(temperature=25, nominal_temperature=25)
analysis_lp_ac = simulator.ac(start_frequency=10@u_Hz, stop_frequency=10@u_kHz, number_of_points=10,  variation='dec')
analysis_lp_tr = simulator.transient(step_time=1@u_us, start_time=100@u_ms, end_time=120@u_ms)



In [ ]:
#load the input mixer schema
kicad_netlist_path = directory_path.joinpath('main', 'HighPass4000.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

#simulate input
schema = parser.build_circuit(ground=5)
schema.include(spice_library['4069UB'])
schema.V('1', '+15V', schema.gnd, 'DC 15')
schema.V('2', '-15V', schema.gnd, 'DC -15')
#schema.V('3', 'NC_01', schema.gnd, 'DC 0V AC 5V SINE(0 5V 1k)')
schema.V('3', 'NC_01', schema.gnd, 'DC 0V AC 5V PULSE(-5V 5V 1u 1u 1u 2m 4m)')

simulator = schema.simulator(temperature=25, nominal_temperature=25)
analysis_hp_ac = simulator.ac(start_frequency=10@u_Hz, stop_frequency=10@u_kHz, number_of_points=10,  variation='dec')
analysis_hp_tr = simulator.transient(step_time=1@u_us, start_time=100@u_ms, end_time=120@u_ms)

In [ ]:
#load the input mixer schema
kicad_netlist_path = directory_path.joinpath('main', 'BandPass900-4000.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

#simulate input
schema = parser.build_circuit(ground=5)
schema.include(spice_library['4069UB'])
schema.V('1', '+15V', schema.gnd, 'DC 15')
schema.V('2', '-15V', schema.gnd, 'DC -15')
#schema.V('3', 'NC_01', schema.gnd, 'DC 0V AC 5V SINE(0 5V 1k)')
schema.V('3', 'NC_01', schema.gnd, 'DC 0V AC 5V PULSE(-5V 5V 1u 1u 1u 2m 4m)')

simulator = schema.simulator(temperature=25, nominal_temperature=25)
analysis_bp_900 = simulator.ac(start_frequency=10@u_Hz, stop_frequency=10@u_kHz, number_of_points=10,  variation='dec')


In [ ]:
#load the input mixer schema
kicad_netlist_path = directory_path.joinpath('main', 'BandPass200-900.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

#simulate input
schema = parser.build_circuit(ground=5)
schema.include(spice_library['4069UB'])
schema.V('1', '+15V', schema.gnd, 'DC 15')
schema.V('2', '-15V', schema.gnd, 'DC -15')
schema.V('3', 'NC_01', schema.gnd, 'DC 0V AC 5V SINE(0 5V 1k)')

simulator = schema.simulator(temperature=25, nominal_temperature=25)
analysis_bp_200 = simulator.ac(start_frequency=10@u_Hz, stop_frequency=10@u_kHz, number_of_points=10,  variation='dec')
analysis_bp_200_tr = simulator.transient(step_time=1@u_us, start_time=100@u_ms, end_time=120@u_ms)

In [ ]:
#plot the results
fig, axis = plt.subplots(figsize=(18, 6))

#plot mixed signal
axis.plot(analysis_lp_tr['/OUT'].abscissa*1000, analysis_lp_tr['/OUT'], c='Orange') 
axis.plot(analysis_hp_tr['/OUT'].abscissa*1000, analysis_hp_tr['/OUT'], c='Green') 
axis.plot(analysis_bp_200_tr['/OUT'].abscissa*1000, analysis_bp_200_tr['/OUT'], c='Blue') 
#axis.plot(analysis_lp_tr['Net-_R2-Pad1_'].abscissa*1000, analysis_lp_tr['Net-_R2-Pad1_'], c='Green') 
#axis.plot(analysis_lp_tr['NC_01'].abscissa*1000, analysis_lp_tr['NC_01'], c='grey') 

axis.grid()
axis.set_xlabel('t [ms]')
axis.set_ylabel('[V]')
axis.set_title('mixed imput saignal', y=-0.2)

plt.show()

In [ ]:
figure, axes = plt.subplots(2, figsize=(20, 10))
plt.title("Bode Diagrams of RLC Filters")
#out = 'Net-_C1-Pad1_'
bode_diagram(axes=axes,
                 frequency=analysis_lp_ac.frequency,
                 gain=np.log10(np.absolute(analysis_lp_ac['/OUT'])),
                 phase=np.angle(analysis_lp_ac['/OUT'], deg=False),
                 marker='.',
                 color='orange',
                 linestyle='-',
    )
bode_diagram(axes=axes,
                 frequency=analysis_hp_ac.frequency,
                 gain=np.log10(np.absolute(analysis_hp_ac['/OUT'])),
                 phase=np.angle(analysis_hp_ac['/OUT'], deg=False),
                 marker='.',
                 color='LightGreen',
                 linestyle='-',
    )
bode_diagram(axes=axes,
                frequency=analysis_bp_900.frequency,
                gain=np.log10(np.absolute(analysis_bp_900['/OUT'])),
                phase=np.angle(analysis_bp_900['/OUT'], deg=False),
                marker='.',
                color='Magenta',
                linestyle='-',
)
bode_diagram(axes=axes,
                frequency=analysis_bp_200.frequency,
                gain=np.log10(np.absolute(analysis_bp_200['/OUT'])),
                phase=np.angle(analysis_bp_200['/OUT'], deg=False),
                marker='.',
                color='Red',
                linestyle='-',
)
    #for axe in axes:
axes[0].axvline(x=194.091394014506, color='Red', linestyle='--')


In [ ]:
#load the input mixer schema
kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

#simulate input
schema = parser.build_circuit(ground=5)
schema.include(spice_library['BC846B'])
schema.include(spice_library['BC556B'])
schema.include(spice_library['CD4069UBhex'])
schema.include(spice_library['D1N4148'])
schema.V('1', '+15V', schema.gnd, 'DC 15')
schema.V('2', '-15V', schema.gnd, 'DC -15')
schema.V('3', '+5V', schema.gnd, 'DC 5')
#schema.V('4', 'INPUT', schema.gnd, 'DC 0V AC 2V PULSE(-5V +5V 0 1m 1m 0.0001m 2.0001m)')
schema.V('4', 'INPUT', schema.gnd, 'DC 0V AC 2V PULSE(-2V +2V 0 0.0001m 5m 0.0001m 5.0002m)')

class VoltageDivider1(SubCircuitFactory):
    __name__ = 'voltage_divider_1'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=100@u_kΩ, w=0.4):
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )

class VoltageDivider2(SubCircuitFactory):
    __name__ = 'voltage_divider_2'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=100@u_kΩ, w=0.4):
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )

for subcircuit in ( VoltageDivider1(R=500@u_kΩ, w=0.999), VoltageDivider2(R=500@u_kΩ, w=0.1) ):
    schema.subcircuit(subcircuit)

simulator = schema.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.transient(step_time=1@u_us, start_time=100@u_ms, end_time=120@u_ms)


### filter sections


## *construction*

for the final circuit input and output buffering and biasing is needed. the buffering is done with opamps. 


first we design the input stage. the input voltage for the X signal should be 10mV and 20mV for the control voltage.


## *references*

- [VCA Techniques Investigated][1] Rod Elliott (ESP)
- [Popular Electronics][2] Keyiing and VCA citcuits for electronic music instruments 
- [VCA-1][3] Thomas Henry CA3080 vca
- [VCA-3][4] René Schmitz differential pair vca
- [#223][5]: Basics of the Gilbert Cell | Analog Multiplier | Mixer | Modulator
- [#224][6]: AM & DSB-SC Modulation with the Gilbert Cell
- [Analog Devices][8]: Analog multiplier application guide 
- [AD633][9]: Datasheet 




[1]: https://static1.squarespace.com/static/5e6e99e8bd2f8a6de8454feb/t/5e77db16e8ff44374c24b124/1584913176675/MBD+BOM+and+build.pdf
[2]: https://www.diystompboxes.com/smfforum/index.php?topic=118397.0
[3]: https://www.diystompboxes.com/smfforum/index.php?topic=99779.0



